In [1]:
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis
from scipy.fft import fft
# import polars as pl
# import kaggle_evaluation.cmi_inference_server

import os

pd.set_option('display.max_columns', None)

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
train.head()

,row_id,sequence_type,sequence_id,sequence_counter,subject,orientation,behavior,phase,gesture,acc_x,acc_y,acc_z,rot_w,rot_x,rot_y,rot_z,thm_1,thm_2,thm_3,thm_4,thm_5,tof_1_v0,tof_1_v1,tof_1_v2,tof_1_v3,tof_1_v4,tof_1_v5,tof_1_v6,tof_1_v7,tof_1_v8,tof_1_v9,tof_1_v10,tof_1_v11,tof_1_v12,tof_1_v13,tof_1_v14,tof_1_v15,tof_1_v16,tof_1_v17,tof_1_v18,tof_1_v19,tof_1_v20,tof_1_v21,tof_1_v22,tof_1_v23,tof_1_v24,tof_1_v25,tof_1_v26,tof_1_v27,tof_1_v28,tof_1_v29,tof_1_v30,tof_1_v31,tof_1_v32,tof_1_v33,tof_1_v34,tof_1_v35,tof_1_v36,tof_1_v37,tof_1_v38,tof_1_v39,tof_1_v40,tof_1_v41,tof_1_v42,tof_1_v43,tof_1_v44,tof_1_v45,tof_1_v46,tof_1_v47,tof_1_v48,tof_1_v49,tof_1_v50,tof_1_v51,tof_1_v52,tof_1_v53,tof_1_v54,tof_1_v55,tof_1_v56,tof_1_v57,tof_1_v58,tof_1_v59,tof_1_v60,tof_1_v61,tof_1_v62,tof_1_v63,tof_2_v0,tof_2_v1,tof_2_v2,tof_2_v3,tof_2_v4,tof_2_v5,tof_2_v6,tof_2_v7,tof_2_v8,tof_2_v9,tof_2_v10,tof_2_v11,tof_2_v12,tof_2_v13,tof_2_v14,tof_2_v15,tof_2_v16,tof_2_v17,tof_2_v18,tof_2_v19,tof_2_v20,tof_2_v21,tof_2_v22,tof_2_v23,tof_2_v24,tof_2_v25,tof_2_v26,tof_2_v27,tof_2_v28,tof_2_v29,tof_2_v30,tof_2_v31,tof_2_v32,tof_2_v33,tof_2_v34,tof_2_v35,tof_2_v36,tof_2_v37,tof_2_v38,tof_2_v39,tof_2_v40,tof_2_v41,tof_2_v42,tof_2_v43,tof_2_v44,tof_2_v45,tof_2_v46,tof_2_v47,tof_2_v48,tof_2_v49,tof_2_v50,tof_2_v51,tof_2_v52,tof_2_v53,tof_2_v54,tof_2_v55,tof_2_v56,tof_2_v57,tof_2_v58,tof_2_v59,tof_2_v60,tof_2_v61,tof_2_v62,tof_2_v63,tof_3_v0,tof_3_v1,tof_3_v2,tof_3_v3,tof_3_v4,tof_3_v5,tof_3_v6,tof_3_v7,tof_3_v8,tof_3_v9,tof_3_v10,tof_3_v11,tof_3_v12,tof_3_v13,tof_3_v14,tof_3_v15,tof_3_v16,tof_3_v17,tof_3_v18,tof_3_v19,tof_3_v20,tof_3_v21,tof_3_v22,tof_3_v23,tof_3_v24,tof_3_v25,tof_3_v26,tof_3_v27,tof_3_v28,tof_3_v29,tof_3_v30,tof_3_v31,tof_3_v32,tof_3_v33,tof_3_v34,tof_3_v35,tof_3_v36,tof_3_v37,tof_3_v38,tof_3_v39,tof_3_v40,tof_3_v41,tof_3_v42,tof_3_v43,tof_3_v44,tof_3_v45,tof_3_v46,tof_3_v47,tof_3_v48,tof_3_v49,tof_3_v50,tof_3_v51,tof_3_v52,tof_3_v53,tof_3_v54,tof_3_v55,tof_3_v56,tof_3_v57,tof_3_v58,tof_3_v59,tof_3_v60,tof_3_v61,tof_3_v62,tof_3_v63,tof_4_v0,tof_4_v1,tof_4_v2,tof_4_v3,tof_4_v4,tof_4_v5,tof_4_v6,tof_4_v7,tof_4_v8,tof_4_v9,tof_4_v10,tof_4_v11,tof_4_v12,tof_4_v13,tof_4_v14,tof_4_v15,tof_4_v16,tof_4_v17,tof_4_v18,tof_4_v19,tof_4_v20,tof_4_v21,tof_4_v22,tof_4_v23,tof_4_v24,tof_4_v25,tof_4_v26,tof_4_v27,tof_4_v28,tof_4_v29,tof_4_v30,tof_4_v31,tof_4_v32,tof_4_v33,tof_4_v34,tof_4_v35,tof_4_v36,tof_4_v37,tof_4_v38,tof_4_v39,tof_4_v40,tof_4_v41,tof_4_v42,tof_4_v43,tof_4_v44,tof_4_v45,tof_4_v46,tof_4_v47,tof_4_v48,tof_4_v49,tof_4_v50,tof_4_v51,tof_4_v52,tof_4_v53,tof_4_v54,tof_4_v55,tof_4_v56,tof_4_v57,tof_4_v58,tof_4_v59,tof_4_v60,tof_4_v61,tof_4_v62,tof_4_v63,tof_5_v0,tof_5_v1,tof_5_v2,tof_5_v3,tof_5_v4,tof_5_v5,tof_5_v6,tof_5_v7,tof_5_v8,tof_5_v9,tof_5_v10,tof_5_v11,tof_5_v12,tof_5_v13,tof_5_v14,tof_5_v15,tof_5_v16,tof_5_v17,tof_5_v18,tof_5_v19,tof_5_v20,tof_5_v21,tof_5_v22,tof_5_v23,tof_5_v24,tof_5_v25,tof_5_v26,tof_5_v27,tof_5_v28,tof_5_v29,tof_5_v30,tof_5_v31,tof_5_v32,tof_5_v33,tof_5_v34,tof_5_v35,tof_5_v36,tof_5_v37,tof_5_v38,tof_5_v39,tof_5_v40,tof_5_v41,tof_5_v42,tof_5_v43,tof_5_v44,tof_5_v45,tof_5_v46,tof_5_v47,tof_5_v48,tof_5_v49,tof_5_v50,tof_5_v51,tof_5_v52,tof_5_v53,tof_5_v54,tof_5_v55,tof_5_v56,tof_5_v57,tof_5_v58,tof_5_v59,tof_5_v60,tof_5_v61,tof_5_v62,tof_5_v63
0,SEQ_000007_000000,Target,SEQ_000007,0,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.683594,6.214844,3.355469,0.134399,-0.355164,-0.447327,-0.809753,28.943842,31.822186,29.553024,28.592863,28.310535,131.0,134.0,132.0,135.0,98.0,74.0,64.0,60.0,-1.0,-1.0,152.0,153.0,141.0,89.0,68.0,63.0,-1.0,-1.0,-1.0,-1.0,169.0,118.0,86.0,73.0,-1.0,-1.0,-1.0,-1.0,-1.0,147.0,110.0,87.0,126.0,-1.0,-1.0,-1.0,-1.0,-1.0,137.0,108.0,115.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,128.0,110.0,129.0,140.0,-1.0,126.0,131.0,-1.0,-1.0,-1.0,108.0,122.0,139.0,113.0,121.0,-1.0,118.0,96.0,-1.0,-1.0,-1.0,-1.0,165.0,124.0,100.0,102.0,119.0,-1.0,-1.0,

## Aggregation
her sequnce için bir satır veri olmalı

In [4]:
# sequence-level metadata
features_df = train[['sequence_id', 'gesture', 'subject', 'sequence_type']].drop_duplicates('sequence_id')

# sequence_counter : Hareketin ne kadar süre devam ettiği (max(sequence_counter)) gibi çıkarımlar için kullanılabilir.
df_len = train.groupby("sequence_id")["sequence_counter"].agg(["count", "max"]).rename(columns={"count": "n_steps", "max": "max_counter"})
features_df = features_df.merge(df_len, on='sequence_id', how='left')

features_df.head()

,sequence_id,gesture,subject,sequence_type,n_steps,max_counter
0,SEQ_000007,Cheek - pinch skin,SUBJ_059520,Target,57,56
1,SEQ_000008,Forehead - pull hairline,SUBJ_020948,Target,68,67
2,SEQ_000013,Cheek - pinch skin,SUBJ_040282,Target,53,52
3,SEQ_000016,Write name on leg,SUBJ_052342,Non-Target,61,60
4,SEQ_000018,Forehead - pull hairline,SUBJ_032165,Target,54,53


In [5]:
# sequence-level metadata
features_test_df = test[['sequence_id', 'subject']].drop_duplicates('sequence_id')

# sequence_counter : Hareketin ne kadar süre devam ettiği (max(sequence_counter)) gibi çıkarımlar için kullanılabilir.
df_len = test.groupby("sequence_id")["sequence_counter"].agg(["count", "max"]).rename(columns={"count": "n_steps", "max": "max_counter"})
features_test_df = features_test_df.merge(df_len, on='sequence_id', how='left')

features_test_df.head()

,sequence_id,subject,n_steps,max_counter
0,SEQ_000001,SUBJ_055840,56,55
1,SEQ_000011,SUBJ_016452,51,50


## acc_x, acc_y, acc_z
#### Yön Bağımsız Özellikler

* acc_mag - magnitude : ivme vektör normu (MUTLAK BÜYÜKLÜK) <br>
Kol farklı açılarda durduğunda x/y/z bileşenleri değişir ama norm sabit kalır

In [6]:
train['acc_mag'] = np.sqrt(train['acc_x'] ** 2 + train['acc_y'] ** 2 + train['acc_z'] ** 2)
train['jerk'] = train['acc_mag'].diff().fillna(0)

test['acc_mag'] = np.sqrt(test['acc_x'] ** 2 + test['acc_y'] ** 2 + test['acc_z'] ** 2)
test['jerk'] = test['acc_mag'].diff().fillna(0)

In [7]:
print(test.shape)
print(train.shape)
print('----------------')
print(features_df.shape)
print(features_test_df.shape)



(107, 338)
(574945, 343)
----------------
(8151, 6)
(2, 4)


In [8]:
import pandas as pd
import numpy as np
from tqdm import tqdm

def extract_acc_features(df):
    features = {}
    for axis in ['x', 'y', 'z']:
        col = f'acc_{axis}'
        features[f'acc_{axis}_mean'] = df[col].mean()
        features[f'acc_{axis}_std'] = df[col].std()
        features[f'acc_{axis}_max'] = df[col].max()
        features[f'acc_{axis}_min'] = df[col].min()
        features[f'acc_{axis}_range'] = df[col].max() - df[col].min()
        features[f'acc_{axis}_skew'] = df[col].skew()
        features[f'acc_{axis}_kurt'] = df[col].kurt()

        # jerk (türevi)
        jerk = np.gradient(df[col])
        features[f'acc_{axis}_jerk_mean'] = jerk.mean()
        features[f'acc_{axis}_jerk_std'] = jerk.std()
        features[f'acc_{axis}_jerk_max'] = jerk.max()
        features[f'acc_{axis}_jerk_min'] = jerk.min()

        # enerji
        features[f'acc_{axis}_energy'] = np.sum(df[col] ** 2)

    # Eksenler arası korelasyon
    features['corr_acc_x_y'] = df['acc_x'].corr(df['acc_y'])
    features['corr_acc_x_z'] = df['acc_x'].corr(df['acc_z'])
    features['corr_acc_y_z'] = df['acc_y'].corr(df['acc_z'])

    # Directional momentum
    features['acc_z_momentum_x_range'] = df['acc_z'].mean() * (df['acc_x'].max() - df['acc_x'].min())

    # Roll tahmini
    roll_angle = np.arctan2(df['acc_y'], df['acc_x'])
    features['roll_angle_mean'] = np.nanmean(roll_angle)
    features['roll_angle_std'] = np.nanstd(roll_angle)
    features['roll_angle_max'] = np.nanmax(roll_angle)
    features['roll_angle_min'] = np.nanmin(roll_angle)

    return features

# sequence_id bazlı özellik çıkarımı
def build_acc_feature_dataframe(df):
    feature_list = []

    for seq_id, seq_df in tqdm(df.groupby("sequence_id")):
        feats = extract_acc_features(seq_df)
        feats["sequence_id"] = seq_id
        feature_list.append(feats)

    features_df = pd.DataFrame(feature_list)
    return features_df
features = build_acc_feature_dataframe(train)  #
features_test = build_acc_feature_dataframe(test)

100%|██████████| 2/2 [00:00<?, ?it/s]


In [9]:
features_df = features_df.merge(features, on='sequence_id', how='left')
print(features_df.head())

features_test_df = features_test_df.merge(features_test, on='sequence_id', how='left')
print(features_test_df.head())


  sequence_id                   gesture      subject sequence_type  n_steps  \
0  SEQ_000007        Cheek - pinch skin  SUBJ_059520        Target       57   
1  SEQ_000008  Forehead - pull hairline  SUBJ_020948        Target       68   
2  SEQ_000013        Cheek - pinch skin  SUBJ_040282        Target       53   
3  SEQ_000016         Write name on leg  SUBJ_052342    Non-Target       61   
4  SEQ_000018  Forehead - pull hairline  SUBJ_032165        Target       54   

   max_counter  acc_x_mean  acc_x_std  acc_x_max  acc_x_min  acc_x_range  \
0           56    6.153098   1.334155   9.015625   3.613281     5.402344   
1           67    3.400506   1.087142   5.906250   1.734375     4.171875   
2           52   -7.058962   1.295184  -3.347656  -9.250000     5.902344   
3           60    5.524654   1.074108   9.378906   3.437500     5.941406   
4           53    5.363715   1.627637   6.832031   1.964844     4.867188   

   acc_x_skew  acc_x_kurt  acc_x_jerk_mean  acc_x_jerk_std  acc_x_je

In [10]:
train['sequence_id'].value_counts().count()

8151

In [11]:
print(features_df.shape)
print(features_test_df.shape)
# sequence_type & gesture yok (TEST)

(8151, 50)
(2, 48)


## rotw, rotx, roty, rotz
Rotation (rot_w, rot_x, rot_y, rot_z) değerleri quaternion biçiminde olduğundan doğrudan yorumlanması zordur.

#### 1. Euler Dönüşümü (Quaternion → Euler Angle)
Quaternions'u yaw, pitch, roll açılarına çevirip bu açıların:

- Ortalama
- Maksimum / Minimum
- Standart sapma
- İlk ve son frame farkı gibi istatistiklerini çıkarabiliriz.

Bu dönüşüm, elin başa göre ne kadar döndüğünü (örneğin yukarı kaldırma, yana eğme vb.) tespit etmemizi sağlar.

#### 2. Orientation Dynamics – Hareket Yönü ve Stabilite
Belirli bir sequence içinde yönelim değişiminin şiddeti:

- Euler açıları veya quaternionlar üzerinden frame-to-frame farkların toplam (orientation_change_magnitude)
- En hızlı dönüş yapılan yön (max_rotation_axis)
- Başlangıç ve bitiş yönü arasındaki açı farkı

#### 3. Orientation Stability
BFRB-like hareketlerde el başta daha sabit durma eğiliminde olabilir:

- Euler açıları veya quaternion'lar için varyans → yaw_var, pitch_var, roll_var
- BFRB olmayanlarda genelde daha fazla hareket olabilir.

#### 4. Dönme Eksenlerinin Normalize Enerjisi
Her eksen için ortalama dönüş "enerjisini" hesaplayabiliriz:

- rot_energy_x = mean(rot_x ** 2)
- Diğer eksenlerle birlikte karşılaştırma → elin hangi eksende daha aktif döndüğü çıkarımı

#### 5. Hand Tilt Classification (Durumsal Binary Feature)
Yarışmadaki birçok hareket için elin başa göre eğimi kritik. Euler dönüşüm sonrası:

El "yukarıda mı" (pitch > threshold) gibi binary feature’lar çıkarılabilir.

#### 6. Zamana Duyarlı Pattern Extraction
Eğer el başa doğru dönerken ve gesture sırasında sabitleniyorsa:

- roll / pitch / yaw eğrileri için low/high frequency sinyal ayrımı 
- FFT ile dominant frekans çıkarımı (düşük frekans: sabit el; yüksek frekans: dalgalanmalı)

In [12]:
import pandas as pd
import numpy as np
from scipy.spatial.transform import Rotation as R

def quaternion_to_euler(df: pd.DataFrame) -> pd.DataFrame:
    quats = df[['rot_x', 'rot_y', 'rot_z', 'rot_w']].to_numpy()
    norms = np.linalg.norm(quats, axis=1)

    valid = norms > 1e-6
    euler = np.full((len(df), 3), np.nan)

    if valid.any():
        r = R.from_quat(quats[valid] / norms[valid, None])
        euler[valid] = r.as_euler('zyx', degrees=True)

    return pd.DataFrame(euler, columns=['yaw', 'pitch', 'roll'], index=df.index)


# ----------------- Euler açılarını ekle -----------------
euler_df = quaternion_to_euler(train)
train = pd.concat([train, euler_df], axis=1)

# ----------------- Sequence‑level özet istatistikler ----
rot_agg = (
    train
    .groupby('sequence_id')
    .agg(
        yaw_mean  = ('yaw',   'mean'),
        yaw_std   = ('yaw',   'std'),
        yaw_min   = ('yaw',   'min'),
        yaw_max   = ('yaw',   'max'),
        yaw_delta = ('yaw',   lambda x: x.iloc[-1]-x.iloc[0]),

        pitch_mean  = ('pitch', 'mean'),
        pitch_std   = ('pitch', 'std'),
        pitch_min   = ('pitch', 'min'),
        pitch_max   = ('pitch', 'max'),
        pitch_delta = ('pitch', lambda x: x.iloc[-1]-x.iloc[0]),

        roll_mean  = ('roll', 'mean'),
        roll_std   = ('roll', 'std'),
        roll_min   = ('roll', 'min'),
        roll_max   = ('roll', 'max'),
        roll_delta = ('roll', lambda x: x.iloc[-1]-x.iloc[0]),
    )
).reset_index()

# ------------- Özellik tablosuna birleştir ----------------
features_df = features_df.merge(rot_agg, on='sequence_id', how='left')

# TEST
# Euler dönüşümünü hesapla ve ana DataFrame'e ekle
euler_df = quaternion_to_euler(test)
test = pd.concat([test, euler_df], axis=1)

# Sequence bazlı özet istatistikler
rot_agg = test.groupby('sequence_id')[['yaw', 'pitch', 'roll']].agg(['mean', 'std', 'min', 'max', lambda x: x.iloc[-1] - x.iloc[0]])
rot_agg.columns = ['_'.join([col[0], col[1] if isinstance(col[1], str) else 'delta']) for col in rot_agg.columns]
rot_agg.reset_index(inplace=True)

# Özellik DataFrame'inize ekleyin
features_test_df = features_test_df.merge(rot_agg, on='sequence_id', how='left')

In [13]:
print(features_df.shape)
print(features_test_df.shape)

(8151, 65)
(2, 63)


In [14]:
def differences(df1, df2):
    # İlk DataFrame'deki sütunlar
    sutunlar_df1 = set(df1.columns)

    # İkinci DataFrame'deki sütunlar
    sutunlar_df2 = set(df2.columns)

    # Sadece df1'de olan sütunlar
    farkli_sutunlar_df1 = sutunlar_df1 - sutunlar_df2

    # Sadece df2'de olan sütunlar
    farkli_sutunlar_df2 = sutunlar_df2 - sutunlar_df1

    # Her iki DataFrame'de de farklı olan sütunlar (birleşim)
    tum_farkli_sutunlar = sutunlar_df1.symmetric_difference(sutunlar_df2)

    print("---")
    print("Sadece birinci DataFrame'de olan sütunlar:", farkli_sutunlar_df1)
    print("---")
    print("Sadece ikinci DataFrame'de olan sütunlar:", farkli_sutunlar_df2)
    print("---")


differences(train,test)
 

---
Sadece birinci DataFrame'de olan sütunlar: {'orientation', 'behavior', 'gesture', 'phase', 'sequence_type'}
---
Sadece ikinci DataFrame'de olan sütunlar: set()
---


In [15]:
import numpy as np
import pandas as pd

def compute_orientation_dynamics(df):
    angles = ['yaw', 'pitch', 'roll']
    results = []

    for seq_id, group in df.groupby('sequence_id'):
        row = {'sequence_id': seq_id}

        diffs = group[angles].diff().abs().sum()
        row['orientation_change_magnitude'] = diffs.sum()

        start = group[angles].iloc[0]
        end = group[angles].iloc[-1]
        delta = (end - start).abs()
        row['orientation_delta_yaw'] = delta['yaw']
        row['orientation_delta_pitch'] = delta['pitch']
        row['orientation_delta_roll'] = delta['roll']

        stds = group[angles].std()
        row['max_rotation_axis'] = stds.idxmax()

        results.append(row)

    orient_df = pd.DataFrame(results)

    # One-hot encoding için tüm olası eksenleri belirle
    orient_df = pd.get_dummies(orient_df, columns=['max_rotation_axis'])

    # Eksik olan sütunları manuel olarak ekle (tutarlılığı sağlamak için)
    for axis in ['max_rotation_axis_pitch', 'max_rotation_axis_roll', 'max_rotation_axis_yaw']:
        if axis not in orient_df.columns:
            orient_df[axis] = False # YOKSA FALSE YAP !!!!!!!!!  SORUN OLABİLİR ?????

    return orient_df


# Kullanım
# train zaten euler açılarını içeriyor varsayımıyla:
orientation_features = compute_orientation_dynamics(train)
orientation_features_test = compute_orientation_dynamics(test)


# features_df ile birleştir
features_df = features_df.merge(orientation_features, on='sequence_id', how='left')
features_test_df = features_test_df.merge(orientation_features_test, on='sequence_id', how='left')

C:\Users\huseyin\AppData\Local\Temp\ipykernel_14544\1478193204.py:22: FutureWarning: The behavior of Series.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  row['max_rotation_axis'] = stds.idxmax()
C:\Users\huseyin\AppData\Local\Temp\ipykernel_14544\1478193204.py:22: FutureWarning: The behavior of Series.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  row['max_rotation_axis'] = stds.idxmax()
C:\Users\huseyin\AppData\Local\Temp\ipykernel_14544\1478193204.py:22: FutureWarning: The behavior of Series.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  row['max_rotation_axis'] = stds.idxmax()
C:\Users\huseyin\AppData\Local\Temp\ipykernel_14544\1478193204.py:22: FutureWarning: The behavior of Series.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future versio

In [16]:
print(features_df.shape)
print(features_test_df.shape)

(8151, 72)
(2, 70)


In [17]:
differences(train,test)

---
Sadece birinci DataFrame'de olan sütunlar: {'orientation', 'behavior', 'gesture', 'phase', 'sequence_type'}
---
Sadece ikinci DataFrame'de olan sütunlar: set()
---


In [18]:
features_test_df.head()

,sequence_id,subject,n_steps,max_counter,acc_x_mean,acc_x_std,acc_x_max,acc_x_min,acc_x_range,acc_x_skew,acc_x_kurt,acc_x_jerk_mean,acc_x_jerk_std,acc_x_jerk_max,acc_x_jerk_min,acc_x_energy,acc_y_mean,acc_y_std,acc_y_max,acc_y_min,acc_y_range,acc_y_skew,acc_y_kurt,acc_y_jerk_mean,acc_y_jerk_std,acc_y_jerk_max,acc_y_jerk_min,acc_y_energy,acc_z_mean,acc_z_std,acc_z_max,acc_z_min,acc_z_range,acc_z_skew,acc_z_kurt,acc_z_jerk_mean,acc_z_jerk_std,acc_z_jerk_max,acc_z_jerk_min,acc_z_energy,corr_acc_x_y,corr_acc_x_z,corr_acc_y_z,acc_z_momentum_x_range,roll_angle_mean,roll_angle_std,roll_angle_max,roll_angle_min,yaw_mean,yaw_std,yaw_min,yaw_max,yaw_<lambda_0>,pitch_mean,pitch_std,pitch_min,pitch_max,pitch_<lambda_0>,roll_mean,roll_std,roll_min,roll_max,roll_<lambda_0>,orientation_change_magnitude,orientation_delta_yaw,orientation_delta_pitch,orientation_delta_roll,max_rotation_axis_yaw,max_rotation_axis_pitch,max_rotation_axis_roll
0,SEQ_000001,SUBJ_055840,56,55,7.750837,1.050640,10.160156,6.132812,4.027344,0.730955,-0.756832,-0.041399,0.466181,1.380859,-2.035156,3424.937988,5.393694,2.353345,7.214844,0.781250,6.433594,-1.222971,-0.319466,0.003453,0.524735,1.666016,-1.800781,1933.751404,-0.093331,2.305147,4.093750,-4.792969,8.886719,-0.837641,-0.385385,-0.003383,0.722215,2.433594,-2.335938,292.741547,-0.856616,-0.737114,0.905823,-0.375878,0.599637,0.265987,0.844646,0.086889,-124.452980,13.759787,-134.790098,-97.201198,-17.249719,-7.726119,4.687385,-17.477808,-0.217066,-11.477719,-89.877106,13.319235,-116.734228,-73.521057,16.508183,238.683552,17.249719,11.477719,16.508183,True,False,False
1,SEQ_000011,SUBJ_016452,51,50,-2.423790,3.534764,9.609375,-4.523438,14.132812,1.986602,2.661071,0.001417,1.358934,4.136719,-6.701172,924.340561,6.621477,4.377906,8.957031,-5.257812,14.214844,-2.030169,2.552805,-0.019072,1.340108,4.884766,-3.732422,3194.344513,-1.357996,4.619076,11.113281,-5.093750,16.207031,1.902655,1.943542,-0.006817,1.871495,8.025391,-6.166016,1160.845123,-0.935494,0.926198,-0.940147,-19.192307,1.605974,0.913583,2.152584,-0.839874,109.841953,128.224303,-175.741916,179.217338,-325.918929,40.400851,15.033985,-21.695485,51.545082,-67.459807,-101.762531,35.094755,-120.052248,-12.430743,47.338983,1694.018850,325.918929,67.459807,47.338983,True,False,False


In [19]:
print(features_df.dtypes)
print('-----------------------------------')
print(features_test_df.dtypes)


sequence_id                 object
gesture                     object
subject                     object
sequence_type               object
n_steps                      int64
                            ...   
orientation_delta_pitch    float64
orientation_delta_roll     float64
max_rotation_axis_pitch       bool
max_rotation_axis_roll        bool
max_rotation_axis_yaw         bool
Length: 72, dtype: object
-----------------------------------
sequence_id                 object
subject                     object
n_steps                      int64
max_counter                  int64
acc_x_mean                 float64
                            ...   
orientation_delta_pitch    float64
orientation_delta_roll     float64
max_rotation_axis_yaw         bool
max_rotation_axis_pitch       bool
max_rotation_axis_roll        bool
Length: 70, dtype: object


In [20]:
def compute_orientation_stability_energy(df):
    result = []

    for seq_id, group in df.groupby('sequence_id'):
        row = {'sequence_id': seq_id}

        # Euler varyansları (stabilite)
        row['yaw_var'] = group['yaw'].var()
        row['pitch_var'] = group['pitch'].var()
        row['roll_var'] = group['roll'].var()

        # Rotasyon enerjisi (eksensel dönüş şiddeti)
        row['rot_energy_x'] = np.mean(group['rot_x'] ** 2)
        row['rot_energy_y'] = np.mean(group['rot_y'] ** 2)
        row['rot_energy_z'] = np.mean(group['rot_z'] ** 2)

        result.append(row)

    return pd.DataFrame(result)

# Train ve test için ayrı ayrı hesapla
orientation_stats_train = compute_orientation_stability_energy(train)
orientation_stats_test = compute_orientation_stability_energy(test)

# features_df ile birleştir
features_df = features_df.merge(orientation_stats_train, on='sequence_id', how='left')
features_test_df = features_test_df.merge(orientation_stats_test, on='sequence_id', how='left')

In [21]:
print(features_df.shape)
print(features_test_df.shape)


(8151, 78)
(2, 76)


## Thermopil

In [22]:
def extract_thermopile_features(df):
    thm_cols = ['thm_1', 'thm_2', 'thm_3', 'thm_4', 'thm_5']
    grouped = df.groupby('sequence_id')
    
    features = pd.DataFrame(index=grouped.size().index)
    
    for col in thm_cols:
        features[f'{col}_mean'] = grouped[col].mean()
        features[f'{col}_std'] = grouped[col].std()
        features[f'{col}_min'] = grouped[col].min()
        features[f'{col}_max'] = grouped[col].max()
        features[f'{col}_range'] = features[f'{col}_max'] - features[f'{col}_min']
        features[f'{col}_delta'] = grouped[col].nth(-1).values - grouped[col].nth(0).values
        features[f'{col}_activity'] = grouped[col].apply(lambda x: np.sum(np.abs(np.diff(x))))
    
    features['thm_mean_mean'] = features[[f'{col}_mean' for col in thm_cols]].mean(axis=1)
    features['thm_std_mean'] = features[[f'{col}_std' for col in thm_cols]].mean(axis=1)
    
    mean_vals = features[[f'{col}_mean' for col in thm_cols]].values
    features['thm_hotspot_index'] = np.argmax(mean_vals, axis=1)
    
    features['thm_symmetry_side'] = grouped.apply(
        lambda x: ((x['thm_1'] + x['thm_5'])/2 - (x['thm_2'] + x['thm_4'])/2).mean())
    features['thm_center_edge_diff'] = grouped.apply(
        lambda x: np.abs(x['thm_3'] - ((x['thm_1'] + x['thm_5']) / 2)).mean())
    
    features.reset_index(inplace=True)
    return features

# Thermopile feature extraction
features_df = features_df.merge(extract_thermopile_features(train), on='sequence_id', how='left')
features_test_df = features_test_df.merge(extract_thermopile_features(test), on='sequence_id', how='left')


C:\Users\huseyin\AppData\Local\Temp\ipykernel_14544\3273308641.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  features['thm_symmetry_side'] = grouped.apply(
C:\Users\huseyin\AppData\Local\Temp\ipykernel_14544\3273308641.py:24: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  features['thm_center_edge_diff'] = grouped.apply(
C:\Users\huseyin\AppData\Local\Temp\ipykernel_14544\3273308641.py:22: Deprecation

In [23]:
features_df.head()

,sequence_id,gesture,subject,sequence_type,n_steps,max_counter,acc_x_mean,acc_x_std,acc_x_max,acc_x_min,acc_x_range,acc_x_skew,acc_x_kurt,acc_x_jerk_mean,acc_x_jerk_std,acc_x_jerk_max,acc_x_jerk_min,acc_x_energy,acc_y_mean,acc_y_std,acc_y_max,acc_y_min,acc_y_range,acc_y_skew,acc_y_kurt,acc_y_jerk_mean,acc_y_jerk_std,acc_y_jerk_max,acc_y_jerk_min,acc_y_energy,acc_z_mean,acc_z_std,acc_z_max,acc_z_min,acc_z_range,acc_z_skew,acc_z_kurt,acc_z_jerk_mean,acc_z_jerk_std,acc_z_jerk_max,acc_z_jerk_min,acc_z_energy,corr_acc_x_y,corr_acc_x_z,corr_acc_y_z,acc_z_momentum_x_range,roll_angle_mean,roll_angle_std,roll_angle_max,roll_angle_min,yaw_mean,yaw_std,yaw_min,yaw_max,yaw_delta,pitch_mean,pitch_std,pitch_min,pitch_max,pitch_delta,roll_mean,roll_std,roll_min,roll_max,roll_delta,orientation_change_magnitude,orientation_delta_yaw,orientation_delta_pitch,orientation_delta_roll,max_rotation_axis_pitch,max_rotation_axis_roll,max_rotation_axis_yaw,yaw_var,pitch_var,roll_var,rot_energy_x,rot_energy_y,rot_energy_z,thm_1_mean,thm_1_std,thm_1_min,thm_1_max,thm_1_range,thm_1_delta,thm_1_activity,thm_2_mean,thm_2_std,thm_2_min,thm_2_max,thm_2_range,thm_2_delta,thm_2_activity,thm_3_mean,thm_3_std,thm_3_min,thm_3_max,thm_3_range,thm_3_delta,thm_3_activity,thm_4_mean,thm_4_std,thm_4_min,thm_4_max,thm_4_range,thm_4_delta,thm_4_activity,thm_5_mean,thm_5_std,thm_5_min,thm_5_max,thm_5_range,thm_5_delta,thm_5_activity,thm_mean_mean,thm_std_mean,thm_hotspot_index,thm_symmetry_side,thm_center_edge_diff
0,SEQ_000007,Cheek - pinch skin,SUBJ_059520,Target,57,56,6.153098,1.334155,9.015625,3.613281,5.402344,-0.545319,-0.391295,0.020285,0.455492,1.339844,-1.302734,2257.733032,3.915570,3.048287,6.519531,-2.019531,8.539062,-1.184870,-0.391547,-0.001028,0.780156,3.542969,-2.566406,1394.261383,5.577782,2.337517,9.792969,1.093750,8.699219,0.586111,-0.897976,0.002981,0.762233,2.105469,-2.357422,2079.347610,0.854289,-0.887190,-0.902468,30.133098,0.445496,0.443626,0.799987,-0.464035,-136.782733,3.686740,-144.000190,-129.923292,3.191850,18.331457,5.661462,7.476183,31.224292,-13.083466,-50.211994,21.639925,-75.969314,-9.568449,9.286070,383.916977,3.191850,13.083466,9.286070,False,True,False,13.592053,32.052153,468.286339,0.082005,0.139978,0.703864,28.630612,0.582076,27.696510,30.543730,2.847219,-0.150585,13.000359,29.571870,2.576799,24.558798,32.010178,7.451380,-1.006186,22.822474,28.576605,1.260533,25.907490,30.090014,4.182524,-0.156540,13.482555,29.177937,0.278147,28.592863,29.761480,1.168617,0.523323,5.657688,27.957446,0.877846,26.047148,29.428299,3.381151,-0.324644,14.319735,28.782894,1.115080,1,-1.080875,0.752568
1,SEQ_000008,Forehead - pull hairline,SUBJ_020948,Target,68,67,3.400506,1.087142,5.906250,1.734375,4.171875,0.146452,-1.069663,-0.028148,0.519107,1.742188,-1.839844,865.499603,5.311179,3.268073,8.667969,-0.222656,8.890625,-0.830467,-1.170949,0.053740,0.672863,2.375000,-2.574219,2633.766251,6.581629,2.475402,11.074219,1.722656,9.351562,0.186346,-1.306469,-0.065545,1.072456,2.833984,-2.986328,3356.163620,0.678886,-0.594269,-0.889196,27.457734,0.817318,0.438181,1.307932,-0.084997,-148.403656,5.623907,-154.261658,-139.255923,-0.328373,3.416037,6.567778,-7.471505,17.183220,0.772071,-43.404627,22.413410,-67.011753,-8.412334,-7.846398,493.946555,0.328373,0.772071,7.846398,False,True,False,31.628330,43.135706,502.360935,0.016126,0.152819,0.767665,30.464309,2.709212,25.985313,32.870808,6.885494,0.901049,19.042677,29.678206,3.885080,23.907709,33.100945,9.193235,0.056095,31.470016,29.179852,3.074828,24.414917,32.316135,7.901218,1.130726,42.520508,30.501325,0.976249,28.755495,31.613327,2.857832,0.866661,12.937668,25.824221,1.165940,24.181562,28.054575,3.873013,-0.130692,25.231140,29.129583,2.362262,3,-1.945501,1.710534
2,SEQ_000013,Cheek - pinch skin,SUBJ_040282,Target,53,52,-7.058962,1.295184,-3.347656,-9.250000,5.902344,0.518519,0.291585,0.034677,0.761798,2.527344,-1.726562,2728.164307,2.346182,2.564639,4.683594,-3.273438,7.957031,-1.445762,0.544205,0.024838,0.732187,2.70

In [24]:
print(features_df.dtypes)
print('---------------------------------------')
print(features_test_df.dtypes)


sequence_id              object
gesture                  object
subject                  object
sequence_type            object
n_steps                   int64
                         ...   
thm_mean_mean           float64
thm_std_mean            float64
thm_hotspot_index         int64
thm_symmetry_side       float64
thm_center_edge_diff    float64
Length: 118, dtype: object
---------------------------------------
sequence_id              object
subject                  object
n_steps                   int64
max_counter               int64
acc_x_mean              float64
                         ...   
thm_mean_mean           float64
thm_std_mean            float64
thm_hotspot_index         int64
thm_symmetry_side       float64
thm_center_edge_diff    float64
Length: 116, dtype: object


## ToF

In [25]:

# # ToF sensör adları
tof_sensor_names = [f"tof_{i}_v{j}" for i in range(1, 6) for j in range(64)]

# def extract_tof_features(df, prefix='tof'):
#     tof_cols = [col for col in df.columns if col.startswith('tof_')]
#     sensor_features = []

#     for sensor_id in range(1, 6):
#         sensor_cols = [f'tof_{sensor_id}_v{i}' for i in range(64)]

#         # NaN ile geçersiz değerleri işaretle
#         df[sensor_cols] = df[sensor_cols].replace(-1, np.nan)

#         # İstatistikler
#         agg = df.groupby('sequence_id')[sensor_cols].agg(['mean', 'std', 'min', 'max', 'median'])
#         agg.columns = [f'{prefix}_sensor{sensor_id}_' + '_'.join(col) for col in agg.columns]

#         # Geçerli piksel oranı
#         valid_ratio = df[sensor_cols].notna().groupby(df['sequence_id']).mean()
#         valid_ratio.columns = [f'{prefix}_sensor{sensor_id}_valid_ratio_pix_{i}' for i in range(64)]

#         # Sensör içi varyans skoru (her frame için varyans, sonra sequence bazında ortalama)
#         frame_var = df[sensor_cols].var(axis=1)
#         frame_var_mean = frame_var.groupby(df['sequence_id']).mean().rename(f'{prefix}_sensor{sensor_id}_frame_variance_mean')

#         sensor_feat = pd.concat([agg, valid_ratio, frame_var_mean], axis=1)
#         sensor_features.append(sensor_feat)

#     # Tüm sensörler için birleştir
#     all_features = pd.concat(sensor_features, axis=1)
#     all_features.reset_index(inplace=True)

#     return all_features

# Geçerli (valid) ToF değerleri üzerinden özet çıkaran fonksiyon
def extract_reduced_tof_features(df):
    features = []
    for sensor_id in range(1, 6):
        pixel_cols = [f"tof_{sensor_id}_v{j}" for j in range(64)]
        pixels = df[pixel_cols].replace(-1, np.nan).to_numpy()

        # İstatistiksel özetler (valid değerler üzerinden)
        mean_vals = np.nanmean(pixels, axis=1)
        std_vals = np.nanstd(pixels, axis=1)
        min_vals = np.nanmin(pixels, axis=1)
        max_vals = np.nanmax(pixels, axis=1)
        range_vals = max_vals - min_vals
        valid_ratio = np.mean(~np.isnan(pixels), axis=1)

        # Merkez ve uç piksel blokları
        center_mean = np.nanmean(pixels[:, 24:40], axis=1)
        edge_mean = np.nanmean(np.hstack([pixels[:, :16], pixels[:, 48:]]), axis=1)
        center_edge_ratio = center_mean / (edge_mean + 1e-6)

        # Sütun adlarını belirle
        sensor_features = pd.DataFrame({
            f'tof_{sensor_id}_mean': mean_vals,
            f'tof_{sensor_id}_std': std_vals,
            f'tof_{sensor_id}_min': min_vals,
            f'tof_{sensor_id}_max': max_vals,
            f'tof_{sensor_id}_range': range_vals,
            f'tof_{sensor_id}_valid_ratio': valid_ratio,
            f'tof_{sensor_id}_center_mean': center_mean,
            f'tof_{sensor_id}_edge_mean': edge_mean,
            f'tof_{sensor_id}_center_edge_ratio': center_edge_ratio,
        })

        features.append(sensor_features)

    return pd.concat(features, axis=1)

# Fonksiyonları daha sonra train ve test dataframe'leri üzerinde çağırabilirsiniz

In [26]:
# Özellik çıkarımı
train_tof_features = extract_reduced_tof_features(train)
test_tof_features = extract_reduced_tof_features(test)

# Sequence ID ile birleştir
train_tof_features['sequence_id'] = train['sequence_id'].values
test_tof_features['sequence_id'] = test['sequence_id'].values

# Sequence-level özet
train_tof_summary = train_tof_features.groupby('sequence_id').mean().reset_index()
test_tof_summary = test_tof_features.groupby('sequence_id').mean().reset_index()

# features_df'ye ekleme
features_df = features_df.merge(train_tof_summary, on='sequence_id', how='left')
features_test_df = features_test_df.merge(test_tof_summary, on='sequence_id', how='left')

C:\Users\huseyin\AppData\Local\Temp\ipykernel_14544\3855581350.py:43: RuntimeWarning: Mean of empty slice
  mean_vals = np.nanmean(pixels, axis=1)
c:\Users\huseyin\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\huseyin\AppData\Local\Temp\ipykernel_14544\3855581350.py:45: RuntimeWarning: All-NaN slice encountered
  min_vals = np.nanmin(pixels, axis=1)
C:\Users\huseyin\AppData\Local\Temp\ipykernel_14544\3855581350.py:46: RuntimeWarning: All-NaN slice encountered
  max_vals = np.nanmax(pixels, axis=1)
C:\Users\huseyin\AppData\Local\Temp\ipykernel_14544\3855581350.py:51: RuntimeWarning: Mean of empty slice
  center_mean = np.nanmean(pixels[:, 24:40], axis=1)
C:\Users\huseyin\AppData\Local\Temp\ipykernel_14544\3855581350.py:52: RuntimeWarning: Mean of empty slice
  edge_mean = np.nanmean(np.hstack([pixels[:, :16], pixels[:, 48:]

In [27]:
print(features_df.shape)
print('---------------------------------------')
print(features_test_df.shape)


(8151, 163)
---------------------------------------
(2, 161)


In [28]:
display(features_df.head())
display(features_test_df.head())

,sequence_id,gesture,subject,sequence_type,n_steps,max_counter,acc_x_mean,acc_x_std,acc_x_max,acc_x_min,acc_x_range,acc_x_skew,acc_x_kurt,acc_x_jerk_mean,acc_x_jerk_std,acc_x_jerk_max,acc_x_jerk_min,acc_x_energy,acc_y_mean,acc_y_std,acc_y_max,acc_y_min,acc_y_range,acc_y_skew,acc_y_kurt,acc_y_jerk_mean,acc_y_jerk_std,acc_y_jerk_max,acc_y_jerk_min,acc_y_energy,acc_z_mean,acc_z_std,acc_z_max,acc_z_min,acc_z_range,acc_z_skew,acc_z_kurt,acc_z_jerk_mean,acc_z_jerk_std,acc_z_jerk_max,acc_z_jerk_min,acc_z_energy,corr_acc_x_y,corr_acc_x_z,corr_acc_y_z,acc_z_momentum_x_range,roll_angle_mean,roll_angle_std,roll_angle_max,roll_angle_min,yaw_mean,yaw_std,yaw_min,yaw_max,yaw_delta,pitch_mean,pitch_std,pitch_min,pitch_max,pitch_delta,roll_mean,roll_std,roll_min,roll_max,roll_delta,orientation_change_magnitude,orientation_delta_yaw,orientation_delta_pitch,orientation_delta_roll,max_rotation_axis_pitch,max_rotation_axis_roll,max_rotation_axis_yaw,yaw_var,pitch_var,roll_var,rot_energy_x,rot_energy_y,rot_energy_z,thm_1_mean,thm_1_std,thm_1_min,thm_1_max,thm_1_range,thm_1_delta,thm_1_activity,thm_2_mean,thm_2_std,thm_2_min,thm_2_max,thm_2_range,thm_2_delta,thm_2_activity,thm_3_mean,thm_3_std,thm_3_min,thm_3_max,thm_3_range,thm_3_delta,thm_3_activity,thm_4_mean,thm_4_std,thm_4_min,thm_4_max,thm_4_range,thm_4_delta,thm_4_activity,thm_5_mean,thm_5_std,thm_5_min,thm_5_max,thm_5_range,thm_5_delta,thm_5_activity,thm_mean_mean,thm_std_mean,thm_hotspot_index,thm_symmetry_side,thm_center_edge_diff,tof_1_mean,tof_1_std,tof_1_min,tof_1_max,tof_1_range,tof_1_valid_ratio,tof_1_center_mean,tof_1_edge_mean,tof_1_center_edge_ratio,tof_2_mean,tof_2_std,tof_2_min,tof_2_max,tof_2_range,tof_2_valid_ratio,tof_2_center_mean,tof_2_edge_mean,tof_2_center_edge_ratio,tof_3_mean,tof_3_std,tof_3_min,tof_3_max,tof_3_range,tof_3_valid_ratio,tof_3_center_mean,tof_3_edge_mean,tof_3_center_edge_ratio,tof_4_mean,tof_4_std,tof_4_min,tof_4_max,tof_4_range,tof_4_valid_ratio,tof_4_center_mean,tof_4_edge_mean,tof_4_center_edge_ratio,tof_5_mean,tof_5_std,tof_5_min,tof_5_max,tof_5_range,tof_5_valid_ratio,tof_5_center_mean,tof_5_edge_mean,tof_5_center_edge_ratio
0,SEQ_000007,Cheek - pinch skin,SUBJ_059520,Target,57,56,6.153098,1.334155,9.015625,3.613281,5.402344,-0.545319,-0.391295,0.020285,0.455492,1.339844,-1.302734,2257.733032,3.915570,3.048287,6.519531,-2.019531,8.539062,-1.184870,-0.391547,-0.001028,0.780156,3.542969,-2.566406,1394.261383,5.577782,2.337517,9.792969,1.093750,8.699219,0.586111,-0.897976,0.002981,0.762233,2.105469,-2.357422,2079.347610,0.854289,-0.887190,-0.902468,30.133098,0.445496,0.443626,0.799987,-0.464035,-136.782733,3.686740,-144.000190,-129.923292,3.191850,18.331457,5.661462,7.476183,31.224292,-13.083466,-50.211994,21.639925,-75.969314,-9.568449,9.286070,383.916977,3.191850,13.083466,9.286070,False,True,False,13.592053,32.052153,468.286339,0.082005,0.139978,0.703864,28.630612,0.582076,27.696510,30.543730,2.847219,-0.150585,13.000359,29.571870,2.576799,24.558798,32.010178,7.451380,-1.006186,22.822474,28.576605,1.260533,25.907490,30.090014,4.182524,-0.156540,13.482555,29.177937,0.278147,28.592863,29.761480,1.168617,0.523323,5.657688,27.957446,0.877846,26.047148,29.428299,3.381151,-0.324644,14.319735,28.782894,1.115080,1,-1.080875,0.752568,101.462627,18.701970,64.122807,137.263158,73.140351,0.491776,105.167919,99.136456,1.084380,131.584300,18.873624,90.719298,163.614035,72.894737,0.516721,120.999208,124.970788,1.225540,88.811663,10.645399,70.000000,113.631579,43.631579,0.659265,77.623725,88.133020,0.992352,88.479068,22.387650,56.771930,141.578947,84.807018,0.636513,98.636698,82.163070,1.142889,125.362645,23.611158,76.105263,165.087719,88.982456,0.600329,136.848416,118.542951,1.170684
1,SEQ_000008,Forehead - pull hairline,SUBJ_020948,Target,68,67,3.400506,1.087142,5.906250,1.734375,4.171875,0.146452,-1.069663,-0.028148,0.519107,1.742188,-1.839844,865.499603,5.311179,3.268073,8.667969,-0.222656,8.890625,-0.830467,-1.170949,0.053740,0.672863,2.375000,-2.574219,2633.76

,sequence_id,subject,n_steps,max_counter,acc_x_mean,acc_x_std,acc_x_max,acc_x_min,acc_x_range,acc_x_skew,acc_x_kurt,acc_x_jerk_mean,acc_x_jerk_std,acc_x_jerk_max,acc_x_jerk_min,acc_x_energy,acc_y_mean,acc_y_std,acc_y_max,acc_y_min,acc_y_range,acc_y_skew,acc_y_kurt,acc_y_jerk_mean,acc_y_jerk_std,acc_y_jerk_max,acc_y_jerk_min,acc_y_energy,acc_z_mean,acc_z_std,acc_z_max,acc_z_min,acc_z_range,acc_z_skew,acc_z_kurt,acc_z_jerk_mean,acc_z_jerk_std,acc_z_jerk_max,acc_z_jerk_min,acc_z_energy,corr_acc_x_y,corr_acc_x_z,corr_acc_y_z,acc_z_momentum_x_range,roll_angle_mean,roll_angle_std,roll_angle_max,roll_angle_min,yaw_mean,yaw_std,yaw_min,yaw_max,yaw_<lambda_0>,pitch_mean,pitch_std,pitch_min,pitch_max,pitch_<lambda_0>,roll_mean,roll_std,roll_min,roll_max,roll_<lambda_0>,orientation_change_magnitude,orientation_delta_yaw,orientation_delta_pitch,orientation_delta_roll,max_rotation_axis_yaw,max_rotation_axis_pitch,max_rotation_axis_roll,yaw_var,pitch_var,roll_var,rot_energy_x,rot_energy_y,rot_energy_z,thm_1_mean,thm_1_std,thm_1_min,thm_1_max,thm_1_range,thm_1_delta,thm_1_activity,thm_2_mean,thm_2_std,thm_2_min,thm_2_max,thm_2_range,thm_2_delta,thm_2_activity,thm_3_mean,thm_3_std,thm_3_min,thm_3_max,thm_3_range,thm_3_delta,thm_3_activity,thm_4_mean,thm_4_std,thm_4_min,thm_4_max,thm_4_range,thm_4_delta,thm_4_activity,thm_5_mean,thm_5_std,thm_5_min,thm_5_max,thm_5_range,thm_5_delta,thm_5_activity,thm_mean_mean,thm_std_mean,thm_hotspot_index,thm_symmetry_side,thm_center_edge_diff,tof_1_mean,tof_1_std,tof_1_min,tof_1_max,tof_1_range,tof_1_valid_ratio,tof_1_center_mean,tof_1_edge_mean,tof_1_center_edge_ratio,tof_2_mean,tof_2_std,tof_2_min,tof_2_max,tof_2_range,tof_2_valid_ratio,tof_2_center_mean,tof_2_edge_mean,tof_2_center_edge_ratio,tof_3_mean,tof_3_std,tof_3_min,tof_3_max,tof_3_range,tof_3_valid_ratio,tof_3_center_mean,tof_3_edge_mean,tof_3_center_edge_ratio,tof_4_mean,tof_4_std,tof_4_min,tof_4_max,tof_4_range,tof_4_valid_ratio,tof_4_center_mean,tof_4_edge_mean,tof_4_center_edge_ratio,tof_5_mean,tof_5_std,tof_5_min,tof_5_max,tof_5_range,tof_5_valid_ratio,tof_5_center_mean,tof_5_edge_mean,tof_5_center_edge_ratio
0,SEQ_000001,SUBJ_055840,56,55,7.750837,1.050640,10.160156,6.132812,4.027344,0.730955,-0.756832,-0.041399,0.466181,1.380859,-2.035156,3424.937988,5.393694,2.353345,7.214844,0.781250,6.433594,-1.222971,-0.319466,0.003453,0.524735,1.666016,-1.800781,1933.751404,-0.093331,2.305147,4.093750,-4.792969,8.886719,-0.837641,-0.385385,-0.003383,0.722215,2.433594,-2.335938,292.741547,-0.856616,-0.737114,0.905823,-0.375878,0.599637,0.265987,0.844646,0.086889,-124.452980,13.759787,-134.790098,-97.201198,-17.249719,-7.726119,4.687385,-17.477808,-0.217066,-11.477719,-89.877106,13.319235,-116.734228,-73.521057,16.508183,238.683552,17.249719,11.477719,16.508183,True,False,False,189.331752,21.971576,177.402017,0.099826,0.399130,0.373045,30.344483,0.527554,29.404165,31.404631,2.000465,0.531841,7.977383,31.704738,0.869936,30.001017,33.143929,3.142912,-2.180260,19.702045,29.959844,0.744500,28.804676,32.595078,3.790401,1.608583,13.600054,33.351966,0.120876,33.141365,33.648617,0.507252,0.064503,3.245098,30.091830,0.713068,28.103935,31.459490,3.355555,-0.439772,12.562983,31.090572,0.595187,3,-2.310195,0.591100,47.112077,14.550525,16.500000,77.964286,61.464286,0.694754,52.090583,43.598407,1.061843,18.893912,7.256463,8.160714,36.321429,28.160714,0.481864,21.394630,17.984281,39754.34948,41.080951,9.350177,21.000000,60.089286,39.089286,0.715681,40.959654,41.378338,0.470091,160.665970,40.281767,109.178571,245.642857,136.464286,0.797433,162.288566,156.776636,1.029469,138.955569,35.814128,70.553571,218.196429,147.642857,0.713449,158.621833,126.234063,1.205740
1,SEQ_000011,SUBJ_016452,51,50,-2.423790,3.534764,9.609375,-4.523438,14.132812,1.986602,2.661071,0.001417,1.358934,4.136719,-6.701172,924.340561,6.621477,4.377906,8.957031,-5.257812,14.214844,-2.030169,2.552805,-0.019072,1.340108,4.884766,-3.732422,3194.344513,-1.357996,4.619076,11.113281,-5.093750,16.207031,1.9

In [29]:
from metric import score
def hierarchical_macro_f1_score(y_true, y_pred,sequence_ids):
    # y_true ve y_pred: pandas.Series veya numpy dizisi (gesture etiketleri)
    
    # DataFrame yapalım
    df_true = pd.DataFrame({'sequence_id': sequence_ids, 'gesture': y_true,})
    df_pred = pd.DataFrame({'sequence_id': sequence_ids, 'gesture': y_pred,})


    return score(df_true, df_pred, row_id_column_name='sequence_id')

In [30]:
print(','.join(f'"{col}"' for col in features_df.drop(columns=['sequence_id', 'gesture', 'subject', 'sequence_type']).columns))


"n_steps","max_counter","acc_x_mean","acc_x_std","acc_x_max","acc_x_min","acc_x_range","acc_x_skew","acc_x_kurt","acc_x_jerk_mean","acc_x_jerk_std","acc_x_jerk_max","acc_x_jerk_min","acc_x_energy","acc_y_mean","acc_y_std","acc_y_max","acc_y_min","acc_y_range","acc_y_skew","acc_y_kurt","acc_y_jerk_mean","acc_y_jerk_std","acc_y_jerk_max","acc_y_jerk_min","acc_y_energy","acc_z_mean","acc_z_std","acc_z_max","acc_z_min","acc_z_range","acc_z_skew","acc_z_kurt","acc_z_jerk_mean","acc_z_jerk_std","acc_z_jerk_max","acc_z_jerk_min","acc_z_energy","corr_acc_x_y","corr_acc_x_z","corr_acc_y_z","acc_z_momentum_x_range","roll_angle_mean","roll_angle_std","roll_angle_max","roll_angle_min","yaw_mean","yaw_std","yaw_min","yaw_max","yaw_delta","pitch_mean","pitch_std","pitch_min","pitch_max","pitch_delta","roll_mean","roll_std","roll_min","roll_max","roll_delta","orientation_change_magnitude","orientation_delta_yaw","orientation_delta_pitch","orientation_delta_roll","max_rotation_axis_pitch","max_rotatio

## HGBM

In [36]:
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import HistGradientBoostingClassifier
import joblib

# IMU özellikleri ve diğer özellikler
imu_features = [col for col in features_df.columns if not col.startswith(('thm_','tof_'))]


# IMU modeli için X ve y (tüm satırlar, sadece IMU özellikleri)
X_imu = features_df[imu_features].drop(columns= ['sequence_id', 'gesture', 'subject', 'sequence_type'])
y_imu = features_df['gesture']

# Full modeli için X ve y (tüm satırlar, full özellikler)
X_full = features_df.drop(columns=['sequence_id', 'gesture', 'subject', 'sequence_type'])
y_full = features_df['gesture']


# {'max_iter': 629, 'max_leaf_nodes': 37, 'learning_rate': 0.1345386179108374, 'min_samples_leaf': 43, 'max_bins': 254, 'l2_regularization': 0.40798224928170634}
def train_and_save_model(X, y, sequence_ids, model_path, model_name="imu"):
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    best_score = 0
    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        val_sequence_ids = sequence_ids.iloc[val_idx]
        model = HistGradientBoostingClassifier()
        if model_name == 'imu':
            model = HistGradientBoostingClassifier(random_state=42, max_iter=720, max_leaf_nodes=48, learning_rate=0.04, min_samples_leaf=36, max_bins=128,n_iter_no_change=30, l2_regularization=0.36, early_stopping= True)
        else:
            model = HistGradientBoostingClassifier()

        model.fit(X_train, y_train)

        val_preds = model.predict(X_val)
        print('val preds : ', val_preds)
        score = hierarchical_macro_f1_score(y_val, val_preds,  val_sequence_ids)

        print(f"Fold {fold} hierarchical macro F1: {score:.4f}")
        print("-----------------------------------------------------------------------")

        if score > best_score:
            best_score = score
            best_model = model
    
    joblib.dump(best_model, f"{model_path}_{model_name}_{best_score:.4f}.pkl")
    print(f"En iyi fold modeli kaydedildi: Metric score = {best_score:.4f}")

# Eğitim
print("TRAINING IMU MODEL...")
train_and_save_model(X_imu, y_imu, features_df['sequence_id'], model_path="model_hgbm", model_name="imu")

print("\nTRAINING FULL MODEL...")
train_and_save_model(X_full, y_full,features_df['sequence_id'], model_path="model_hgbm", model_name="full")

TRAINING IMU MODEL...
val preds :  ['Forehead - pull hairline' 'Wave hello' 'Wave hello' ...
 'Forehead - pull hairline' 'Text on phone' 'Cheek - pinch skin']
Fold 0 hierarchical macro F1: 0.6780
-----------------------------------------------------------------------
val preds :  ['Above ear - pull hair' 'Write name on leg' 'Eyebrow - pull hair' ...
 'Pull air toward your face' 'Above ear - pull hair'
 'Forehead - pull hairline']
Fold 1 hierarchical macro F1: 0.6697
-----------------------------------------------------------------------
val preds :  ['Eyebrow - pull hair' 'Neck - scratch' 'Forehead - pull hairline' ...
 'Above ear - pull hair' 'Cheek - pinch skin' 'Eyelash - pull hair']
Fold 2 hierarchical macro F1: 0.6654
-----------------------------------------------------------------------
val preds :  ['Text on phone' 'Pull air toward your face' 'Forehead - scratch' ...
 'Eyebrow - pull hair' 'Neck - scratch' 'Pinch knee/leg skin']
Fold 3 hierarchical macro F1: 0.6801
------------

KeyboardInterrupt: 

### LGBM

In [34]:
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
import joblib

# IMU özellikleri ve diğer özellikler
imu_features = [col for col in features_df.columns if not col.startswith(('thm_','tof_'))]


# IMU modeli için X ve y (tüm satırlar, sadece IMU özellikleri)
X_imu = features_df[imu_features].drop(columns= ['sequence_id', 'gesture', 'subject', 'sequence_type'])
y_imu = features_df['gesture']

# Full modeli için X ve y (tüm satırlar, full özellikler)
X_full = features_df.drop(columns=['sequence_id', 'gesture', 'subject', 'sequence_type'])
y_full = features_df['gesture']


# {'max_iter': 629, 'max_leaf_nodes': 37, 'learning_rate': 0.1345386179108374, 'min_samples_leaf': 43, 'max_bins': 254, 'l2_regularization': 0.40798224928170634}
def train_and_save_model(X, y, sequence_ids, model_path, model_name="imu"):
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    best_score = 0
    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        val_sequence_ids = sequence_ids.iloc[val_idx]

        model = LGBMClassifier(verbose = -1)
        model.fit(X_train, y_train)

        val_preds = model.predict(X_val)
        score = hierarchical_macro_f1_score(y_val, val_preds,  val_sequence_ids)

        print(f"Fold {fold} hierarchical macro F1: {score:.4f}")
        print("-----------------------------------------------------------------------")

        if score > best_score:
            best_score = score
            best_model = model
    
    joblib.dump(best_model, f"{model_path}_{model_name}_{best_score:.4f}.pkl")
    print(f"En iyi fold modeli kaydedildi: Metric score = {best_score:.4f}")

# Eğitim
print("TRAINING IMU MODEL...")
train_and_save_model(X_imu, y_imu, features_df['sequence_id'], model_path="lgbm", model_name="imu")

print("\nTRAINING FULL MODEL...")
train_and_save_model(X_full, y_full,features_df['sequence_id'], model_path="lgbm", model_name="full")

TRAINING IMU MODEL...
Fold 0 hierarchical macro F1: 0.6828
-----------------------------------------------------------------------
Fold 1 hierarchical macro F1: 0.6720
-----------------------------------------------------------------------
Fold 2 hierarchical macro F1: 0.6679
-----------------------------------------------------------------------
Fold 3 hierarchical macro F1: 0.6769
-----------------------------------------------------------------------
Fold 4 hierarchical macro F1: 0.6807
-----------------------------------------------------------------------
En iyi fold modeli kaydedildi: Metric score = 0.6828

TRAINING FULL MODEL...
Fold 0 hierarchical macro F1: 0.7888
-----------------------------------------------------------------------
Fold 1 hierarchical macro F1: 0.7851
-----------------------------------------------------------------------
Fold 2 hierarchical macro F1: 0.7807
-----------------------------------------------------------------------
Fold 3 hierarchical macro F1:

## CATBOOST

In [35]:
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostClassifier
import joblib

# IMU özellikleri ve diğer özellikler
imu_features = [col for col in features_df.columns if not col.startswith(('thm_','tof_'))]


# IMU modeli için X ve y (tüm satırlar, sadece IMU özellikleri)
X_imu = features_df[imu_features].drop(columns= ['sequence_id', 'gesture', 'subject', 'sequence_type'])
y_imu = features_df['gesture']

# Full modeli için X ve y (tüm satırlar, full özellikler)
X_full = features_df.drop(columns=['sequence_id', 'gesture', 'subject', 'sequence_type'])
y_full = features_df['gesture']


# {'max_iter': 629, 'max_leaf_nodes': 37, 'learning_rate': 0.1345386179108374, 'min_samples_leaf': 43, 'max_bins': 254, 'l2_regularization': 0.40798224928170634}
def train_and_save_model(X, y, sequence_ids, model_path, model_name="imu"):
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    best_score = 0
    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        val_sequence_ids = sequence_ids.iloc[val_idx]

        model = CatBoostClassifier(verbose = 0)
        model.fit(X_train, y_train)
        val_preds = model.predict(X_val)
        val_preds_flat = val_preds.flatten() if hasattr(val_preds, 'flatten') else val_preds
        score = hierarchical_macro_f1_score(y_val, val_preds_flat,  val_sequence_ids)

        print(f"Fold {fold} hierarchical macro F1: {score:.4f}")
        print("-----------------------------------------------------------------------")
        
        if score > best_score:
            best_score = score
            best_model = model
    
    joblib.dump(best_model, f"{model_path}_{model_name}_{best_score:.4f}.pkl")
    print(f"En iyi fold modeli kaydedildi: Metric score = {best_score:.4f}")

# Eğitim
print("TRAINING IMU MODEL...")
train_and_save_model(X_imu, y_imu, features_df['sequence_id'], model_path="model_cb", model_name="imu")

print("\nTRAINING FULL MODEL...")
train_and_save_model(X_full, y_full,features_df['sequence_id'], model_path="model_cb", model_name="full")

TRAINING IMU MODEL...
Fold 0 hierarchical macro F1: 0.6795
-----------------------------------------------------------------------
Fold 1 hierarchical macro F1: 0.6877
-----------------------------------------------------------------------
Fold 2 hierarchical macro F1: 0.6795
-----------------------------------------------------------------------
Fold 3 hierarchical macro F1: 0.6889
-----------------------------------------------------------------------
Fold 4 hierarchical macro F1: 0.6818
-----------------------------------------------------------------------
En iyi fold modeli kaydedildi: Metric score = 0.6889

TRAINING FULL MODEL...
Fold 0 hierarchical macro F1: 0.7920
-----------------------------------------------------------------------
Fold 1 hierarchical macro F1: 0.8019
-----------------------------------------------------------------------
Fold 2 hierarchical macro F1: 0.7891
-----------------------------------------------------------------------
Fold 3 hierarchical macro F1:

## XGBOOST

In [36]:
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
import joblib
from sklearn.preprocessing import LabelEncoder

# IMU özellikleri ve diğer özellikler
imu_features = [col for col in features_df.columns if not col.startswith(('thm_','tof_'))]


# IMU modeli için X ve y (tüm satırlar, sadece IMU özellikleri)
X_imu = features_df[imu_features].drop(columns= ['sequence_id', 'gesture', 'subject', 'sequence_type'])
y_imu = features_df['gesture']

le = LabelEncoder()
y_encoded_imu = le.fit_transform(y_imu)

# Full modeli için X ve y (tüm satırlar, full özellikler)
X_full = features_df.drop(columns=['sequence_id', 'gesture', 'subject', 'sequence_type'])
y_full = features_df['gesture']
y_encoded_full = le.fit_transform(y_full)


# {'max_iter': 629, 'max_leaf_nodes': 37, 'learning_rate': 0.1345386179108374, 'min_samples_leaf': 43, 'max_bins': 254, 'l2_regularization': 0.40798224928170634}
def train_and_save_model(X, y_np, sequence_ids, model_path, model_name="imu"):
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    best_score, best_model = 0, None

    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y_np)):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val   = y_np[train_idx],   y_np[val_idx]      # <‑‑ düz indeksleme

        model = XGBClassifier(n_estimators=600, tree_method="hist", verbose=0)
        model.fit(X_train, y_train)

        val_preds = model.predict(X_val)
        val_preds_decoded = le.inverse_transform(val_preds)
        # print('pred decoded: ',val_preds_decoded)
        # print('pred y_val: ',y_val)
        y_val_decoded = le.inverse_transform(y_val)

        score = hierarchical_macro_f1_score(y_val_decoded, val_preds_decoded, sequence_ids.iloc[val_idx])

        print(f"{model_name.upper()} | Fold {fold} F1: {score:.4f}")
        if score > best_score:
            best_score, best_model = score, model

    joblib.dump(best_model, f"{model_path}_{model_name}_{best_score:.4f}.pkl")
    print(f"Saved best {model_name} model – F1={best_score:.4f}")


# Eğitim
print("TRAINING IMU MODEL...")
joblib.dump(le, "enc_imu.pkl") 
train_and_save_model(X_imu, y_encoded_imu, features_df['sequence_id'], model_path="model_xgb", model_name="imu")


print("\nTRAINING FULL MODEL...")
train_and_save_model(X_full, y_encoded_full,features_df['sequence_id'], model_path="model_xgb", model_name="full")

TRAINING IMU MODEL...


/Users/teklif_bilisim/.pyenv/versions/stabil/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [16:16:28] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "verbose" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


IMU | Fold 0 F1: 0.6921


/Users/teklif_bilisim/.pyenv/versions/stabil/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [16:16:50] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "verbose" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


IMU | Fold 1 F1: 0.6717


...

FULL | Fold 2 F1: 0.7866


/Users/teklif_bilisim/.pyenv/versions/stabil/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [16:19:46] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "verbose" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


FULL | Fold 3 F1: 0.7866


/Users/teklif_bilisim/.pyenv/versions/stabil/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [16:20:17] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "verbose" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


FULL | Fold 4 F1: 0.7820
Saved best full model – F1=0.7912


In [ ]:
test[test['tof_1_v0'] == -1]

,row_id,sequence_id,sequence_counter,subject,acc_x,acc_y,acc_z,rot_w,rot_x,rot_y,rot_z,thm_1,thm_2,thm_3,thm_4,thm_5,tof_1_v0,tof_1_v1,tof_1_v2,tof_1_v3,tof_1_v4,tof_1_v5,tof_1_v6,tof_1_v7,tof_1_v8,tof_1_v9,tof_1_v10,tof_1_v11,tof_1_v12,tof_1_v13,tof_1_v14,tof_1_v15,tof_1_v16,tof_1_v17,tof_1_v18,tof_1_v19,tof_1_v20,tof_1_v21,tof_1_v22,tof_1_v23,tof_1_v24,tof_1_v25,tof_1_v26,tof_1_v27,tof_1_v28,tof_1_v29,tof_1_v30,tof_1_v31,tof_1_v32,tof_1_v33,tof_1_v34,tof_1_v35,tof_1_v36,tof_1_v37,tof_1_v38,tof_1_v39,tof_1_v40,tof_1_v41,tof_1_v42,tof_1_v43,tof_1_v44,tof_1_v45,tof_1_v46,tof_1_v47,tof_1_v48,tof_1_v49,tof_1_v50,tof_1_v51,tof_1_v52,tof_1_v53,tof_1_v54,tof_1_v55,tof_1_v56,tof_1_v57,tof_1_v58,tof_1_v59,tof_1_v60,tof_1_v61,tof_1_v62,tof_1_v63,tof_2_v0,tof_2_v1,tof_2_v2,tof_2_v3,tof_2_v4,tof_2_v5,tof_2_v6,tof_2_v7,tof_2_v8,tof_2_v9,tof_2_v10,tof_2_v11,tof_2_v12,tof_2_v13,tof_2_v14,tof_2_v15,tof_2_v16,tof_2_v17,tof_2_v18,tof_2_v19,tof_2_v20,tof_2_v21,tof_2_v22,tof_2_v23,tof_2_v24,tof_2_v25,tof_2_v26,tof_2_v27,tof_2_v28,tof_2_v29,tof_2_v30,tof_2_v31,tof_2_v32,tof_2_v33,tof_2_v34,tof_2_v35,tof_2_v36,tof_2_v37,tof_2_v38,tof_2_v39,tof_2_v40,tof_2_v41,tof_2_v42,tof_2_v43,tof_2_v44,tof_2_v45,tof_2_v46,tof_2_v47,tof_2_v48,tof_2_v49,tof_2_v50,tof_2_v51,tof_2_v52,tof_2_v53,tof_2_v54,tof_2_v55,tof_2_v56,tof_2_v57,tof_2_v58,tof_2_v59,tof_2_v60,tof_2_v61,tof_2_v62,tof_2_v63,tof_3_v0,tof_3_v1,tof_3_v2,tof_3_v3,tof_3_v4,tof_3_v5,tof_3_v6,tof_3_v7,tof_3_v8,tof_3_v9,tof_3_v10,tof_3_v11,tof_3_v12,tof_3_v13,tof_3_v14,tof_3_v15,tof_3_v16,tof_3_v17,tof_3_v18,tof_3_v19,tof_3_v20,tof_3_v21,tof_3_v22,tof_3_v23,tof_3_v24,tof_3_v25,tof_3_v26,tof_3_v27,tof_3_v28,tof_3_v29,tof_3_v30,tof_3_v31,tof_3_v32,tof_3_v33,tof_3_v34,tof_3_v35,tof_3_v36,tof_3_v37,tof_3_v38,tof_3_v39,tof_3_v40,tof_3_v41,tof_3_v42,tof_3_v43,tof_3_v44,tof_3_v45,tof_3_v46,tof_3_v47,tof_3_v48,tof_3_v49,tof_3_v50,tof_3_v51,tof_3_v52,tof_3_v53,tof_3_v54,tof_3_v55,tof_3_v56,tof_3_v57,tof_3_v58,tof_3_v59,tof_3_v60,tof_3_v61,tof_3_v62,tof_3_v63,tof_4_v0,tof_4_v1,tof_4_v2,tof_4_v3,tof_4_v4,tof_4_v5,tof_4_v6,tof_4_v7,tof_4_v8,tof_4_v9,tof_4_v10,tof_4_v11,tof_4_v12,tof_4_v13,tof_4_v14,tof_4_v15,tof_4_v16,tof_4_v17,tof_4_v18,tof_4_v19,tof_4_v20,tof_4_v21,tof_4_v22,tof_4_v23,tof_4_v24,tof_4_v25,tof_4_v26,tof_4_v27,tof_4_v28,tof_4_v29,tof_4_v30,tof_4_v31,tof_4_v32,tof_4_v33,tof_4_v34,tof_4_v35,tof_4_v36,tof_4_v37,tof_4_v38,tof_4_v39,tof_4_v40,tof_4_v41,tof_4_v42,tof_4_v43,tof_4_v44,tof_4_v45,tof_4_v46,tof_4_v47,tof_4_v48,tof_4_v49,tof_4_v50,tof_4_v51,tof_4_v52,tof_4_v53,tof_4_v54,tof_4_v55,tof_4_v56,tof_4_v57,tof_4_v58,tof_4_v59,tof_4_v60,tof_4_v61,tof_4_v62,tof_4_v63,tof_5_v0,tof_5_v1,tof_5_v2,tof_5_v3,tof_5_v4,tof_5_v5,tof_5_v6,tof_5_v7,tof_5_v8,tof_5_v9,tof_5_v10,tof_5_v11,tof_5_v12,tof_5_v13,tof_5_v14,tof_5_v15,tof_5_v16,tof_5_v17,tof_5_v18,tof_5_v19,tof_5_v20,tof_5_v21,tof_5_v22,tof_5_v23,tof_5_v24,tof_5_v25,tof_5_v26,tof_5_v27,tof_5_v28,tof_5_v29,tof_5_v30,tof_5_v31,tof_5_v32,tof_5_v33,tof_5_v34,tof_5_v35,tof_5_v36,tof_5_v37,tof_5_v38,tof_5_v39,tof_5_v40,tof_5_v41,tof_5_v42,tof_5_v43,tof_5_v44,tof_5_v45,tof_5_v46,tof_5_v47,tof_5_v48,tof_5_v49,tof_5_v50,tof_5_v51,tof_5_v52,tof_5_v53,tof_5_v54,tof_5_v55,tof_5_v56,tof_5_v57,tof_5_v58,tof_5_v59,tof_5_v60,tof_5_v61,tof_5_v62,tof_5_v63,acc_mag,jerk,yaw,pitch,roll
3,SEQ_000001_000003,SEQ_000001,3,SUBJ_055840,9.773438,1.355469,-4.371094,0.387756,-0.531982,-0.634033,-0.405823,31.043678,32.963718,29.661499,33.603333,29.308136,-1.0,-1.0,-1.0,214.0,228.0,228.0,163.0,145.0,-1.0,200.0,193.0,197.0,215.0,224.0,180.0,149.0,-1.0,183.0,175.0,176.0,201.0,211.0,202.0,161.0,182.0,165.0,166.0,181.0,187.0,207.0,-1.0,181.0,151.0,151.0,162.0,171.0,185.0,189.0,-1.0,-1.0,137.0,144.0,148.0,161.0,177.0,152.0,-1.0,-1.0,123.0,132.0,141.0,129.0,120.0,93.0,77.0,68.0,113.0,109.0,97.0,85.0,77.0,68.0,61.0,54.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,58.0,57.0,53.0,-1.0,-1.0,-1.0,-1.

## XGB with OPTUNA

In [31]:
import optuna
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
import joblib

imu_features = [col for col in features_df.columns if not col.startswith(('thm_','tof_'))]

X_imu = features_df[imu_features].drop(columns=['sequence_id', 'gesture', 'subject', 'sequence_type'])
y_imu = features_df['gesture']


le = LabelEncoder()
y_encoded_imu = le.fit_transform(y_imu)

X_full = features_df.drop(columns=['sequence_id', 'gesture', 'subject', 'sequence_type'])
y_full = features_df['gesture']

y_encoded_full = le.fit_transform(y_full)


def objective(trial, X, y, sequence_ids):
    params = {
        "max_depth":        trial.suggest_int("max_depth", 6, 11),
        "min_child_weight": trial.suggest_float("min_child_weight", 0.1, 5.0, log=True),
        "gamma":            trial.suggest_float("gamma", 0.0, 2.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 0.9),
        "subsample":        trial.suggest_float("subsample", 0.7, 0.9),
        "reg_lambda":       trial.suggest_float("reg_lambda", 0.1, 10, log=True),
        "reg_alpha":        trial.suggest_float("reg_alpha", 0.0, 2.0),
        "learning_rate":    trial.suggest_float("learning_rate", 0.02, 0.08, log=True),
        "n_estimators":     trial.suggest_int("n_estimators", 600, 1600, step = 100),
        "max_delta_step":   trial.suggest_int("max_delta_step", 0, 4),
        "objective":        "multi:softprob",
        "num_class":        len(np.unique(y)),
        "verbosity":        0,
        "random_state":     42,
        "tree_method":      "hist",          # varsa GPU: "gpu_hist"
    }

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = []

    for train_idx, val_idx in skf.split(X, y):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]          # <-- düzeltildi
        val_seq_ids    = sequence_ids.iloc[val_idx]

        model = XGBClassifier(**params)
        model.fit(X_train, y_train)

        preds = model.predict(X_val)
        preds_decoded = le.inverse_transform(preds)
        # print('pred decoded: ',val_preds_decoded)
        # print('pred y_val: ',y_val)
        y_val_decoded = le.inverse_transform(y_val)
        score = hierarchical_macro_f1_score(y_val_decoded, preds_decoded, val_seq_ids)
        scores.append(score)

    return np.mean(scores)


def train_and_save_model_with_optuna(X, y, sequence_ids, model_path, model_name="IMU", n_trials=100):
    study = optuna.create_study(direction="maximize")
    study.optimize(lambda trial: objective(trial, X, y, sequence_ids), n_trials=n_trials)

    print(f"Best trial params: {study.best_trial.params}")
    print(f"Best trial score: {study.best_trial.value:.4f}")

    # En iyi parametrelerle modeli tekrar tüm veride eğit
    best_params = study.best_trial.params
    best_params["random_state"] = 42

    best_model = XGBClassifier(**best_params)
    best_model.fit(X, y)

    joblib.dump(best_model, f"{model_path}_{model_name}_{study.best_trial.value:.4f}.pkl")
    print(f"En iyi model kaydedildi: Metric score = {study.best_trial.value:.4f}")

# Eğitim  
print("TRAINING IMU MODEL WITH OPTUNA...")
train_and_save_model_with_optuna(X_imu, y_encoded_imu, features_df['sequence_id'], model_path="model_xgb_optuna", model_name="imu", n_trials=100)

print("\nTRAINING FULL MODEL WITH OPTUNA...")
train_and_save_model_with_optuna(X_full, y_encoded_full, features_df['sequence_id'], model_path="model_xgb_optuna", model_name="full", n_trials=100)

/Users/teklif_bilisim/.pyenv/versions/stabil/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-06-13 20:12:52,861] A new study created in memory with name: no-name-77d7aa5f-01d3-42a0-b47a-7e5325dc7f1a


TRAINING IMU MODEL WITH OPTUNA...


[I 2025-06-13 20:15:36,355] Trial 0 finished with value: 0.6801031006131991 and parameters: {'max_depth': 9, 'min_child_weight': 3.709758215822068, 'gamma': 1.091490597134129, 'colsample_bytree': 0.7874834568406306, 'subsample': 0.7550573771466548, 'reg_lambda': 1.4143985697424324, 'reg_alpha': 1.0568475273442546, 'learning_rate': 0.028458398207429878, 'n_estimators': 1600, 'max_delta_step': 0}. Best is trial 0 with value: 0.6801031006131991.
[I 2025-06-13 20:18:35,252] Trial 1 finished with value: 0.6792086888987721 and parameters: {'max_depth': 6, 'min_child_weight': 0.2518139952407489, 'gamma': 0.9177120332390445, 'colsample_bytree': 0.8528418409414399, 'subsample': 0.8043533952359618, 'reg_lambda': 0.6290724107863149, 'reg_alpha': 0.3905589445017583, 'learning_rate': 0.020386001295025127, 'n_estimators': 900, 'max_delta_step': 1}. Best is trial 0 with value: 0.6801031006131991.
[I 2025-06-13 20:21:59,649] Trial 2 finished with value: 0.6784767726877511 and parameters: {'max_depth':

Best trial params: {'max_depth': 6, 'min_child_weight': 0.5317377149836044, 'gamma': 0.1423566259801424, 'colsample_bytree': 0.8779908348339797, 'subsample': 0.8248012746289748, 'reg_lambda': 5.465529619795072, 'reg_alpha': 1.4782738380064977, 'learning_rate': 0.0751908813470053, 'n_estimators': 600, 'max_delta_step': 2}
Best trial score: 0.6848


[I 2025-06-14 01:51:04,914] A new study created in memory with name: no-name-a9645700-3824-4841-b4fc-839dd654aae6


En iyi model kaydedildi: Metric score = 0.6848

TRAINING FULL MODEL WITH OPTUNA...


[I 2025-06-14 01:55:25,675] Trial 0 finished with value: 0.7743748806176114 and parameters: {'max_depth': 10, 'min_child_weight': 0.14182414211690095, 'gamma': 1.1456346169013396, 'colsample_bytree': 0.6160581643647094, 'subsample': 0.7059688617053544, 'reg_lambda': 0.3610686133893097, 'reg_alpha': 1.6936958711693948, 'learning_rate': 0.07647426479718301, 'n_estimators': 1500, 'max_delta_step': 0}. Best is trial 0 with value: 0.7743748806176114.
[I 2025-06-14 02:00:59,854] Trial 1 finished with value: 0.7756155479992434 and parameters: {'max_depth': 9, 'min_child_weight': 2.312533472112632, 'gamma': 1.5963509167594456, 'colsample_bytree': 0.6123882446886946, 'subsample': 0.7623220363389281, 'reg_lambda': 6.385853053748329, 'reg_alpha': 0.15898941538652744, 'learning_rate': 0.0207460062698945, 'n_estimators': 600, 'max_delta_step': 0}. Best is trial 1 with value: 0.7756155479992434.
[I 2025-06-14 02:05:53,441] Trial 2 finished with value: 0.7780453036014862 and parameters: {'max_depth':

Best trial params: {'max_depth': 9, 'min_child_weight': 4.634232191055498, 'gamma': 0.13068307704960275, 'colsample_bytree': 0.8527034745262859, 'subsample': 0.7657175762606607, 'reg_lambda': 0.8767113203132394, 'reg_alpha': 0.2579842694602782, 'learning_rate': 0.030734261285110462, 'n_estimators': 1300, 'max_delta_step': 2}
Best trial score: 0.7893
En iyi model kaydedildi: Metric score = 0.7893


## CATBOOST With OPTUNA

In [31]:
import optuna
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostClassifier


import joblib

imu_features = [col for col in features_df.columns if not col.startswith(('thm_','tof_'))]

X_imu = features_df[imu_features].drop(columns=['sequence_id', 'gesture', 'subject', 'sequence_type'])
y_imu = features_df['gesture']

X_full = features_df.drop(columns=['sequence_id', 'gesture', 'subject', 'sequence_type'])
y_full = features_df['gesture']


def objective(trial, X, y, sequence_ids):
    params = {
        # === Temel yapı ===
        "task_type": "GPU",
        "devices": "0",
        "depth":               trial.suggest_int("depth", 6, 10),  # Çok derinlik overfit yapar, 6–8 arası genelde ideal
        "learning_rate":       trial.suggest_float("learning_rate", 0.03, 0.08, log=True),  # 0.05 civarı genelde en iyi

        # === Regularization ===
        "l2_leaf_reg":         trial.suggest_float("l2_leaf_reg", 3.0, 12.0),  # L2 düzenleme
        "random_strength":     trial.suggest_float("random_strength", 0.5, 3.0),  # Rastgelelik, genel performansı arttırır
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.1, 1.5),  # Bayes bagging, stabiliteyi artırır

        # === Sampling & büyüme ===
        #"subsample":           trial.suggest_float("subsample", 0.8, 1.0),  # Düşük alt örnekleme istatistiksel özelliklerde riskli
        #"colsample_bylevel":   trial.suggest_float("colsample_bylevel", 0.6, 0.95),
        "grow_policy":         trial.suggest_categorical("grow_policy", ["SymmetricTree"]),  # Depthwise gereksiz karmaşıklık katıyor

        # === Split hassasiyeti ===
        "border_count":        trial.suggest_int("border_count", 96, 192),  # ToF gibi analog yoğunluklarda yüksek değer faydalı
        "min_data_in_leaf":    trial.suggest_int("min_data_in_leaf", 10, 30),  # Leaf overfit'i kırar

        # === Eğitim kontrol ===
        "iterations":          trial.suggest_int("iterations", 1400, 2800, step = 200),  # 0.05 learning_rate için 1800+ ideal
        "early_stopping_rounds": 100,
        "loss_function":       "MultiClass",
        "eval_metric":         "TotalF1",
        "auto_class_weights":  "Balanced",
        "random_seed":         42,
        "verbose":             0
    }
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = []
    for train_idx, val_idx in skf.split(X, y):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]          # <-- düzeltildi
        val_seq_ids    = sequence_ids.iloc[val_idx]

        model = CatBoostClassifier(**params)
        model.fit(X_train, y_train)

        val_preds = model.predict(X_val)
        val_preds_flat = val_preds.flatten() if hasattr(val_preds, 'flatten') else val_preds
        score = hierarchical_macro_f1_score(y_val, val_preds_flat, val_seq_ids)
        scores.append(score)

    return np.mean(scores)


def train_and_save_model_with_optuna(X, y, sequence_ids, model_path, model_name="IMU", n_trials=100):
    study = optuna.create_study(direction="maximize")
    study.optimize(lambda trial: objective(trial, X, y, sequence_ids), n_trials=n_trials)

    print(f"Best trial params: {study.best_trial.params}")
    print(f"Best trial score: {study.best_trial.value:.4f}")

    # En iyi parametrelerle modeli tekrar tüm veride eğit
    best_params = study.best_trial.params
    best_params["random_state"] = 42

    best_model = CatBoostClassifier(**best_params)
    best_model.fit(X, y)
    ,

    joblib.dump(best_model, f"{model_path}_{model_name}_{study.best_trial.value:.4f}.pkl")
    print(f"En iyi model kaydedildi: Metric score = {study.best_trial.value:.4f}")
 
# Eğitim  
#print("TRAINING IMU MODEL WITH OPTUNA...")
#train_and_save_model_with_optuna(X_imu, y_imu, features_df['sequence_id'], model_path="model_cb_optuna", model_name="imu", n_trials=100)

print("\nTRAINING FULL MODEL WITH OPTUNA...")
train_and_save_model_with_optuna(X_full, y_full, features_df['sequence_id'], model_path="model_cb_optuna", model_name="full", n_trials=100)

c:\Users\huseyin\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-06-15 11:31:47,862] A new study created in memory with name: no-name-02f91358-11a5-4ac1-9cd4-3ad8c520cb8a



TRAINING FULL MODEL WITH OPTUNA...


[I 2025-06-15 11:35:31,725] Trial 0 finished with value: 0.7900933402661228 and parameters: {'depth': 8, 'learning_rate': 0.07992244392323199, 'l2_leaf_reg': 6.0429747828337845, 'random_strength': 1.9732482015051698, 'bagging_temperature': 1.3594021764984867, 'grow_policy': 'SymmetricTree', 'border_count': 135, 'min_data_in_leaf': 29, 'iterations': 1400}. Best is trial 0 with value: 0.7900933402661228.
[I 2025-06-15 11:38:27,443] Trial 1 finished with value: 0.790157816726034 and parameters: {'depth': 7, 'learning_rate': 0.046030032231654645, 'l2_leaf_reg': 5.287070036532777, 'random_strength': 2.5874625661594832, 'bagging_temperature': 1.456361195813098, 'grow_policy': 'SymmetricTree', 'border_count': 143, 'min_data_in_leaf': 27, 'iterations': 1800}. Best is trial 1 with value: 0.790157816726034.
[I 2025-06-15 11:44:53,307] Trial 2 finished with value: 0.7899055598702988 and parameters: {'depth': 8, 'learning_rate': 0.06244953611024144, 'l2_leaf_reg': 7.743922850430787, 'random_streng

Best trial params: {'depth': 7, 'learning_rate': 0.06562893192522636, 'l2_leaf_reg': 7.382560221980152, 'random_strength': 2.573389458706914, 'bagging_temperature': 1.1415454751177083, 'grow_policy': 'SymmetricTree', 'border_count': 122, 'min_data_in_leaf': 24, 'iterations': 2600}
Best trial score: 0.7957
0:	learn: 2.8152968	total: 204ms	remaining: 8m 50s
1:	learn: 2.7660062	total: 415ms	remaining: 8m 59s
2:	learn: 2.7098519	total: 624ms	remaining: 9m
3:	learn: 2.6558566	total: 835ms	remaining: 9m 1s
4:	learn: 2.6161625	total: 1.02s	remaining: 8m 51s
5:	learn: 2.5682736	total: 1.22s	remaining: 8m 46s
6:	learn: 2.5268785	total: 1.41s	remaining: 8m 41s
7:	learn: 2.4823106	total: 1.59s	remaining: 8m 36s
8:	learn: 2.4558621	total: 1.78s	remaining: 8m 33s
9:	learn: 2.4213634	total: 1.98s	remaining: 8m 31s
10:	learn: 2.3881177	total: 2.19s	remaining: 8m 36s
11:	learn: 2.3541768	total: 2.4s	remaining: 8m 38s
12:	learn: 2.3262009	total: 2.6s	remaining: 8m 36s
13:	learn: 2.2971701	total: 2.78s	

## LGBM OPTUNA

In [ ]:
import optuna
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMClassifier
import joblib

imu_features = [col for col in features_df.columns if not col.startswith(('thm_','tof_'))]

X_imu = features_df[imu_features].drop(columns=['sequence_id', 'gesture', 'subject', 'sequence_type'])
y_imu = features_df['gesture']

X_full = features_df.drop(columns=['sequence_id', 'gesture', 'subject', 'sequence_type'])
y_full = features_df['gesture']


def objective(trial, X, y, sequence_ids):
    params = {
        # --- core tree shape --------------------------------------------------
        "boosting_type": trial.suggest_categorical("boosting_type", ["gbdt", "dart"]),
        "num_leaves":     trial.suggest_int("num_leaves", 32, 256, step=8),
        "max_depth":      trial.suggest_int("max_depth", -1, 14),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 10, 80),
        "min_sum_hessian_in_leaf": trial.suggest_float(
            "min_sum_hessian_in_leaf", 1e-3, 50.0, log=True
        ),

        # --- regularisation / penalties --------------------------------------
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-3, 5.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-3, 5.0, log=True),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.55, 0.95),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.55, 0.95),
        "bagging_freq":     trial.suggest_int("bagging_freq", 1, 8),

        # --- learning dynamics -----------------------------------------------
        "learning_rate":  trial.suggest_float("learning_rate", 0.02, 0.1, log=True),
        "n_estimators":   trial.suggest_int("n_estimators", 1000, 3000, step = 200),
        # DART‑only drop parameters (ignored for GBDT)
        "drop_rate":      trial.suggest_float("drop_rate", 0.05, 0.35),
        "skip_drop":      trial.suggest_float("skip_drop", 0.3, 0.8),

        # --- class & loss -----------------------------------------------------
        "objective":      "multiclass",
        "num_class":      18,
        "metric":         "multi_logloss",   # surrogate during tuning
        "force_row_wise": True,              # faster on ~8k rows
        "verbosity":      -1,
        "seed":           42,
    }
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = []
    for train_idx, val_idx in skf.split(X, y):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]          # <-- düzeltildi
        val_seq_ids    = sequence_ids.iloc[val_idx]

        model = LGBMClassifier(**params)
        model.fit(X_train, y_train)

        val_preds = model.predict(X_val)
        score = hierarchical_macro_f1_score(y_val, val_preds, val_seq_ids)
        scores.append(score)

    return np.mean(scores)


def train_and_save_model_with_optuna(X, y, sequence_ids, model_path, model_name="IMU", n_trials=100):
    study = optuna.create_study(direction="maximize")
    study.optimize(lambda trial: objective(trial, X, y, sequence_ids), n_trials=n_trials)

    print(f"Best trial params: {study.best_trial.params}")
    print(f"Best trial score: {study.best_trial.value:.4f}")

    # En iyi parametrelerle modeli tekrar tüm veride eğit
    best_params = study.best_trial.params
    best_params["random_state"] = 42

    best_model = LGBMClassifier(**best_params)
    best_model.fit(X, y)

    joblib.dump(best_model, f"{model_path}_{model_name}_{study.best_trial.value:.4f}.pkl")
    print(f"En iyi model kaydedildi: Metric score = {study.best_trial.value:.4f}")

# Eğitim  
print("TRAINING IMU MODEL WITH OPTUNA...")
train_and_save_model_with_optuna(X_imu, y_imu, features_df['sequence_id'], model_path="model_lgbm_optuna", model_name="imu", n_trials=100)

print("\nTRAINING FULL MODEL WITH OPTUNA...")
train_and_save_model_with_optuna(X_full, y_full, features_df['sequence_id'], model_path="model_lgbm_optuna", model_name="full", n_trials=100)

[I 2025-06-14 20:39:52,842] A new study created in memory with name: no-name-80726501-3d92-4c22-b23e-3f05a5084d97


TRAINING IMU MODEL WITH OPTUNA...


In [51]:
import sklearn
print(sklearn.__version__)

1.2.2


In [ ]:
pd.set_option('display.max_columns', None)

In [44]:
print(features_df.columns.tolist())

['sequence_id', 'gesture', 'subject', 'sequence_type', 'n_steps', 'max_counter', 'acc_x_mean', 'acc_x_std', 'acc_x_max', 'acc_x_min', 'acc_x_range', 'acc_x_skew', 'acc_x_kurt', 'acc_x_jerk_mean', 'acc_x_jerk_std', 'acc_x_jerk_max', 'acc_x_jerk_min', 'acc_x_energy', 'acc_y_mean', 'acc_y_std', 'acc_y_max', 'acc_y_min', 'acc_y_range', 'acc_y_skew', 'acc_y_kurt', 'acc_y_jerk_mean', 'acc_y_jerk_std', 'acc_y_jerk_max', 'acc_y_jerk_min', 'acc_y_energy', 'acc_z_mean', 'acc_z_std', 'acc_z_max', 'acc_z_min', 'acc_z_range', 'acc_z_skew', 'acc_z_kurt', 'acc_z_jerk_mean', 'acc_z_jerk_std', 'acc_z_jerk_max', 'acc_z_jerk_min', 'acc_z_energy', 'corr_acc_x_y', 'corr_acc_x_z', 'corr_acc_y_z', 'acc_z_momentum_x_range', 'roll_angle_mean', 'roll_angle_std', 'roll_angle_max', 'roll_angle_min', 'yaw_mean', 'yaw_std', 'yaw_min', 'yaw_max', 'yaw_<lambda_0>', 'pitch_mean', 'pitch_std', 'pitch_min', 'pitch_max', 'pitch_<lambda_0>', 'roll_mean', 'roll_std', 'roll_min', 'roll_max', 'roll_<lambda_0>', 'orientation_